In [71]:
# Célula 1: Instalação de Bibliotecas e Importações

print("Iniciando a instalação e importação das bibliotecas necessárias...")

# 1. Instalação das Bibliotecas
# A biblioteca 'google-generativeai' nos permite usar a IA Gemini do Google.
print("Instalando google-generativeai...")
!pip install --no-cache-dir google-generativeai

# A biblioteca 'google-maps-services-python' nos permite pegar informações do Google Maps, como avaliações de lugares.
# Cuidado: O nome do pacote para instalar é 'google-maps-services-python', mas o módulo que importamos é 'googlemaps'.
print("Instalando google-maps-services-python...")
!pip install googlemaps

# Se você planeja usar pandas para análise de dados tabulares posteriormente, pode instalar aqui.
# print("Instalando pandas (opcional, para análise de dados tabulares)...")
# !pip install --no-cache-dir pandas

print("\nInstalação de bibliotecas concluída!")

# 2. Importação das Bibliotecas
# Importamos as bibliotecas que acabamos de instalar e outras bibliotecas nativas do Python que vamos usar.

import google.generativeai as genai     # Para interagir com a IA Gemini
from google.colab import userdata       # Para pegar as chaves de API de forma segura no Colab
from googlemaps import Client as GoogleMapsClient # Para interagir com o Google Maps Places API

import re                               # Biblioteca para trabalhar com expressões regulares (limpeza de texto)
from collections import Counter         # Para contar a frequência de itens em uma lista (ex: sentimentos)

print("Bibliotecas importadas com sucesso!")

Iniciando a instalação e importação das bibliotecas necessárias...
Instalando google-generativeai...
Instalando google-maps-services-python...

Instalação de bibliotecas concluída!
Bibliotecas importadas com sucesso!


In [59]:
# Célula 1: Instalação de Bibliotecas

# A biblioteca 'google-generativeai' nos permite usar a IA Gemini do Google.
!pip install google-generativeai



In [62]:
!pip install googlemaps

In [58]:
# Configura a API Key do Google Gemini

import os
from google.colab import userdata

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')



In [73]:
# Configura o cliente da SDK do Gemini

from google import genai as ggai

client = ggai.Client()

MODEL_ID = "gemini-2.0-flash"

In [61]:
# Instalar Framework ADK de agentes do Google ################################################
!pip install -q google-adk

In [63]:
from googlemaps import Client as GoogleMapsClient


In [64]:
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types  # Para criar conteúdos (Content e Part)
from datetime import date
from datetime import datetime  # Importa a classe 'datetime' do módulo 'datetime' para registrar a data e hora.
import random  # Importa o módulo 'random' para gerar números aleatórios e fazer escolhas aleatórias.
import time    # Importa o módulo 'time' para controlar o tempo, como pausas.
import textwrap # Para formatar melhor a saída de texto
from IPython.display import display, HTML, Markdown # Para exibir texto formatado no Colab
import requests # Para fazer requisições HTTP
import warnings

warnings.filterwarnings("ignore")

# INICIO DO PROJETO

In [74]:
# Célula 2: Configuração das Chaves de API

print("Tentando configurar as chaves de API e inicializar os serviços do Google...")

# Pegamos as chaves que você configurou nos "Segredos" do Colab.
Maps_API_KEY = userdata.get('Maps_API_KEY')
GEMINI_API_KEY = userdata.get('GOOGLE_API_KEY')

# Verificamos se as chaves foram configuradas corretamente.
if not Maps_API_KEY:
    print("ERRO: Maps_API_KEY não encontrada. Por favor, configure sua chave do Google Maps nos 'Segredos' do Colab.")
    print("Siga as instruções na seção '2. Configurando suas Chaves de API'.")
elif not GEMINI_API_KEY:
    print("ERRO: GEMINI_API_KEY não encontrada. Por favor, configure sua chave do Google AI Studio nos 'Segredos' do Colab.")
    print("Siga as instruções na seção '2. Configurando suas Chaves de API'.")
else:
    try:
        # Inicializamos o cliente do Google Maps com sua chave.
        gmaps = GoogleMapsClient(Maps_API_KEY)
        print("Cliente do Google Maps inicializado com sucesso!")

        # Configuramos a IA Gemini com sua chave.
        genai.configure(api_key=GEMINI_API_KEY)
        # Escolhemos o modelo 'gemini-pro' para análise de texto.
        model = genai.GenerativeModel('gemini-pro')
        print("Modelo Gemini ('gemini-pro') inicializado com sucesso!")

        print("\nChaves de API configuradas e serviços do Google inicializados!")
    except Exception as e:
        print(f"ERRO: Ocorreu um problema ao inicializar os serviços do Google: {e}")
        print("Verifique suas chaves de API e sua conexão com a internet.")

Tentando configurar as chaves de API e inicializar os serviços do Google...
Cliente do Google Maps inicializado com sucesso!
Modelo Gemini ('gemini-pro') inicializado com sucesso!

Chaves de API configuradas e serviços do Google inicializados!


In [78]:
# Célula 3: Funções do Projeto (Mais uma atualização!)

# (Mantenha as funções 'buscar_reviews' e 'limpar_texto' como estão)

# Função para buscar avaliações (reviews) de um local usando o ID do local
def buscar_reviews(place_id):
    """
    Busca as avaliações de um local específico no Google Maps.
    Recebe: place_id (ID único de um local no Google Maps).
    Retorna: Uma lista de textos de avaliações.
    """
    try:
        place_details = gmaps.place(place_id=place_id, fields=['reviews'])
        if 'reviews' in place_details['result']:
            return [review['text'] for review in place_details['result']['reviews']]
        else:
            print(f"Aviso: Não foram encontradas avaliações para o Place ID: {place_id}")
            return []
    except Exception as e:
        print(f"Erro ao buscar reviews para {place_id}: {e}")
        return []

# Função para limpar o texto das avaliações
def limpar_texto(texto):
    """
    Limpa o texto de uma avaliação, removendo emojis e caracteres especiais.
    Recebe: texto (string da avaliação).
    Retorna: O texto limpo.
    """
    texto_limpo = re.sub(r'[^\w\s]+', '', texto)
    texto_limpo = re.sub(r'[\U0001F600-\U0001F64F\U0001F300-\U0001F5FF\U0001F680-\U0001F6FF\u2600-\u26FF\u2700-\u27BF]', '', texto_limpo)
    return texto_limpo.strip()

# Função para extrair palavras-chave e sentimento (mantém como está, já atualizada)
def extrair_palavras_chave_e_sentimento(texto):
    """
    Usa o modelo Gemini para extrair palavras-chave e determinar o sentimento
    do texto em relação a essas palavras.
    """
    prompt = f"""
    Analise a seguinte avaliação em português.
    1. Identifique 3 a 5 palavras-chave mais relevantes que descrevam o tema principal da avaliação.
    2. Determine o sentimento geral da avaliação como 'positivo', 'negativo' ou 'neutro'.

    Formato de saída (apenas JSON):
    {{
      "sentimento": "positivo|negativo|neutro",
      "palavras_chave": ["palavra1", "palavra2", "palavra3"]
    }}

    Avaliação: "{texto}"
    """
    try:
        response = model.generate_content(prompt)
        response_text = response.text.strip().lower()

        print(response_text)

        if response_text.startswith('```json'):
            response_text = response_text[len('```json'):]
        if response_text.endswith('```'):
            response_text = response_text[:-len('```')]


        data = json.loads(response_text)

        sentimento = data.get('sentimento', 'erro').strip()
        palavras_chave = data.get('palavras_chave', [])

        if sentimento not in ['positivo', 'negativo', 'neutro']:
            sentimento = 'neutro'

        return {'sentimento': sentimento, 'palavras_chave': palavras_chave}

    except Exception as e:
        print(f"Erro na análise de sentimento/palavras-chave para o texto '{texto[:70]}...': {e}")
        return {'sentimento': 'erro', 'palavras_chave': []}

# --- NOVA FUNÇÃO: AGENTE DE BUSCA DE PLACE ID ---
def buscar_place_id_por_nome(nome_do_local, cidade_uf=None):
    """
    Busca o Place ID de um local pelo seu nome, opcionalmente com cidade e estado.
    Recebe: nome_do_local (string, ex: "Parque da Cidade").
            cidade_uf (string opcional, ex: "Joinville, SC").
    Retorna: O place_id do local ou None se não encontrado.
    """
    query = nome_do_local
    if cidade_uf:
        query = f"{nome_do_local}, {cidade_uf}"

    try:
        # Usa a API de 'find_place' do Google Maps para buscar por nome.
        # O campo 'place_id' é o que queremos.
        # 'inputtype='textquery' é o tipo de busca.
        response = gmaps.find_place(query, 'textquery', fields=['place_id', 'name', 'formatted_address'])

        if response['candidates']:
            # Pega o primeiro resultado como o mais relevante.
            candidato = response['candidates'][0]
            print(f"Local encontrado: '{candidato.get('name')}' - Endereço: '{candidato.get('formatted_address')}'")
            return candidato['place_id']
        else:
            print(f"Não foi encontrado nenhum local com o nome: '{nome_do_local}'")
            return None
    except Exception as e:
        print(f"Erro ao buscar Place ID para '{nome_do_local}': {e}")
        return None

import json # Garante que 'json' está importado para a função 'extrair_palavras_chave_e_sentimento'

print("Todas as funções auxiliares (incluindo o agente de palavras-chave e busca de Place ID) foram definidas!")

Todas as funções auxiliares (incluindo o agente de palavras-chave e busca de Place ID) foram definidas!


In [ ]:
from logging import info
# Célula 4: Análise Principal do Local (Atualizada para usar o Agente de Busca de Local)

nome_do_local_desejado =  ""

while nome_do_local_desejado != "sair":
    # --- NOVIDADE: Pedimos ao usuário o nome do local ---

    nome_do_local_desejado = input("Digite o nome do local público que deseja analisar (ex: Parque da Cidade): ")
    if nome_do_local_desejado.lower() == "sair":
        print("Saindo do programa. Até a próxima!")
        break

    cidade_uf_do_local = input("Digite a cidade e UF do local (ex: Joinville, SC): ")
    if cidade_uf_do_local.lower() == "sair":
        print("Saindo do programa. Até a próxima!")
        break
    # Usamos o AGENTE DE BUSCA DE PLACE ID para encontrar o ID automaticamente!
    place_id = buscar_place_id_por_nome(nome_do_local_desejado, cidade_uf_do_local)

    if place_id: # Se o Place ID foi encontrado...
        print(f"\nIniciando a análise de sentimento para o local com ID: {place_id}")

        reviews = buscar_reviews(place_id)

        if reviews:
            resultados_analise = []
            todas_palavras_chave = []

            print(f"\nTotal de avaliações encontradas: {len(reviews)}")
            print("\n--- Analisando cada avaliação com o Agente PlaceVibes AI ---")

            for i, review in enumerate(reviews):
                texto_limpo = limpar_texto(review)

                if texto_limpo:
                    resultado = extrair_palavras_chave_e_sentimento(texto_limpo)
                    resultados_analise.append(resultado)

                    sentimento = resultado['sentimento']
                    palavras_chave = resultado['palavras_chave']

                    todas_palavras_chave.extend(palavras_chave)

                    print(f"Avaliação {i+1}: '{texto_limpo[:70]}...'")
                    print(f"  -> Sentimento: {sentimento.capitalize()}, Palavras-chave: {', '.join(palavras_chave) if palavras_chave else 'Nenhuma'}")
                else:
                    print(f"Avaliação {i+1}: Vazia após a limpeza, pulando.")

            # Resumo dos Sentimentos e Palavras-Chave (mantém como está)
            if resultados_analise:
                sentimentos_para_contagem = [res['sentimento'] for res in resultados_analise if res['sentimento'] != 'erro']
                contagem_sentimentos = Counter(sentimentos_para_contagem)
                total_reviews_analisadas = len(sentimentos_para_contagem)

                print("\n--- Resumo do Sentimento Geral ---")
                if total_reviews_analisadas > 0:
                    for sentimento, count in contagem_sentimentos.items():
                        porcentagem = (count / total_reviews_analisadas) * 100
                        print(f"{sentimento.capitalize()}: {count} ({porcentagem:.2f}%)")
                else:
                    print("Nenhum sentimento válido foi analisado.")

                print("\n--- Palavras-Chave Mais Frequentes ---")
                if todas_palavras_chave:
                    contagem_palavras_chave = Counter(todas_palavras_chave)
                    for palavra, count in contagem_palavras_chave.most_common(10):
                        print(f"- {palavra.capitalize()}: {count} ocorrências")
                else:
                    print("Nenhuma palavra-chave relevante foi extraída.")

                print("\n--- Diagnóstico Qualitativo Inicial ---")
                if 'positivo' in contagem_sentimentos and contagem_sentimentos['positivo'] > contagem_sentimentos.get('negativo', 0) * 2:
                    print("💪 O local parece ter uma recepção predominantemente POSITIVA! Ótimo para o turismo e satisfação pública. As palavras-chave podem indicar os pontos fortes.")
                elif 'negativo' in contagem_sentimentos and contagem_sentimentos['negativo'] > contagem_sentimentos.get('positivo', 0):
                    print("⚠️ Atenção: Há um volume significativo de sentimento NEGATIVO. O local pode precisar de melhorias. As palavras-chave podem apontar os problemas.")
                elif 'neutro' in contagem_sentimentos and contagem_sentimentos['neutro'] > total_reviews_analisadas * 0.5:
                    print("🤔 O sentimento predominante é NEUTRO. Isso pode indicar falta de elementos que gerem forte conexão ou emoção. Verifique as palavras-chave para entender o que está faltando.")
                else:
                    print("📊 Sentimento balanceado ou diversificado. Uma análise mais profunda dos comentários individuais e palavras-chave seria útil.")

            else:
                print("Não foi possível analisar nenhum sentimento das avaliações do local.")
        else:
            print("Não foi possível obter avaliações para este local ou ocorreu um erro na busca inicial.")
    else:
        print("Análise cancelada. Não foi possível encontrar o Place ID para o local fornecido.")
